In [449]:
import os
import numpy as np
import pandas as pd
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn import preprocessing
from sklearn import model_selection, preprocessing, metrics
from sklearn.metrics import mean_squared_error

In [450]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [451]:
train_df['time'] = np.log1p(train_df['time'])

In [452]:
train_y = train_df.pop('time')

In [453]:
train_y = train_y.to_frame(name='time')

In [454]:
#combine data
ntrain = train_df.shape[0]
ntest = test_df.shape[0]
all_data = pd.concat((train_df,test_df)).reset_index(drop=True)
print('the size of all_data is:{}'.format(all_data.shape))

the size of all_data is:(500, 14)


In [455]:
all_data = all_data.drop(['id'],axis=1)

In [456]:
all_data = all_data.drop(['random_state'],axis=1)

In [457]:
all_data = all_data.drop(['l1_ratio'],axis=1)

In [458]:
all_data = all_data.drop(['alpha'],axis=1)

In [459]:
all_data = all_data.drop(['n_clusters_per_class'],axis=1)

In [460]:
all_data = all_data.drop(['flip_y'],axis=1)

In [461]:
all_data = all_data.drop(['scale'],axis=1)

In [462]:
#encode nominal features
label_encoder=[]
encoded_set = np.empty(all_data.shape)
for col in range(all_data.shape[1]):
    encoder=None
    if all_data.iloc[:,col].dtype==object:
        encoder=preprocessing.LabelEncoder()
        tmp = encoder.fit_transform(all_data.iloc[:,col])
        all_data.iloc[:,col] = tmp
    else:  
        encoded_set[:,col]=all_data.iloc[:,col]
    label_encoder.append(encoder)

In [463]:
all_data['new'] = all_data.apply(lambda row: row['max_iter'] * row['n_samples']*row['n_features'], axis=1)
all_data.drop(['max_iter'],axis = 1,inplace = True) 
all_data.drop(['n_samples'],axis = 1,inplace = True) 
all_data.drop(['n_features'],axis = 1,inplace = True) 

In [464]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [465]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
pca.fit(train)
train.shape
train= pd.DataFrame(train)

In [466]:
import lightgbm as lgb
lgbmodel = lgb.LGBMRegressor()

In [467]:
model = lgbmodel.fit(train,train_y)

In [468]:
print(model)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [469]:
test_pred = lgbmodel.predict(test)

In [470]:
print (test_pred)

[1.08059298 2.22013314 0.31346539 1.09356255 1.09780477 2.18851658
 1.31859231 0.49305852 2.78657222 0.31604314 1.84159861 2.32662337
 0.51737677 3.28107539 0.24865641 0.56332885 0.63950708 1.52711395
 1.5880243  0.8651693  0.34046043 0.43235872 0.53532688 0.4678179
 0.88809714 1.0605923  0.77981767 1.76933177 1.14452036 1.47712032
 2.91245073 1.38688011 0.47988297 2.14612552 2.24244583 0.774038
 1.39465924 0.94730647 0.59790073 1.13169235 1.70530734 2.4021357
 0.85066042 1.47091684 0.70791287 1.43042697 1.58459205 0.55357602
 2.01459339 0.61249127 0.19030869 2.65442895 0.19442851 2.00528123
 1.93571938 0.20315385 0.88140684 0.62010934 0.58541827 0.76233869
 0.80551725 1.81252201 0.3074816  2.30558437 1.27413125 2.45705407
 0.75591044 0.33525344 2.26298978 0.57215166 0.27136017 0.59490367
 2.56080602 0.20571709 1.7548946  1.52047205 1.21769714 1.63487247
 1.49315318 0.88188893 2.30558437 1.33697911 1.49364862 0.77721071
 0.73210767 1.39602031 1.99598079 0.29776549 1.9089707  0.70495589

In [471]:
ans = test_pred

In [472]:
ans[:10]

array([1.08059298, 2.22013314, 0.31346539, 1.09356255, 1.09780477,
       2.18851658, 1.31859231, 0.49305852, 2.78657222, 0.31604314])

In [473]:
back = np.expm1(ans)

In [474]:
back[:10]

array([ 1.94642621,  8.20855678,  0.36815811,  1.98488898,  1.99757842,
        7.92196827,  2.7381555 ,  0.63731634, 15.22530755,  0.37168942])

In [475]:
test = pd.DataFrame(data=back)

In [476]:
test.columns = ['time']

In [477]:
test.head()

,time
0,1.946426
1,8.208557
2,0.368158
3,1.984889
4,1.997578


In [29]:
test.to_csv('5001kaggletreel1_4.58.csv',index_label = "id",header = "time")